# Merge and pivot

In [1]:
import addutils.toc ; addutils.toc.js(ipy_notebook=True)

In this tutorial we are going to see how to combine `pandas` datastructures together.

In [2]:
import numpy as np
import pandas as pd
from IPython.display import (display, HTML)
from addutils import side_by_side2
from addutils import css_notebook
from pandas.io.data import DataFrame, read_csv
css_notebook()

## 1 Concat

`pandas.concat` concatenates two (or more) pandas objects along a particular axis (default is the vertical one).

In [3]:
data = np.array([[-1, -2, 4, 3],[3,4, -4, -3], [2, -5, 3, 2], [2, -5, 3, 2]])
df1 = pd.DataFrame(data, columns = ['a','b','c','d'])
df2 = pd.DataFrame(np.random.randint(5, size=(3,6)) + 0.5, columns = list('fedcba'))
HTML(side_by_side2(df1, df2))

When gluing together multiple DataFrames, you have a choice of how to handle the other axes (other than the one being concatenated). This can be done by defining `join`:

In [4]:
HTML(side_by_side2(pd.concat([df1, df2]), pd.concat([df1, df2], join='inner')))

It is possible to create a Multi-index DataFrame setting the 'keys' parameter.

In [5]:
pd.concat([df1, df2], keys = ('First', 'Second'))

a    b    c    d    e    f
First  0 -1.0 -2.0  4.0  3.0  NaN  NaN
       1  3.0  4.0 -4.0 -3.0  NaN  NaN
       2  2.0 -5.0  3.0  2.0  NaN  NaN
       3  2.0 -5.0  3.0  2.0  NaN  NaN
Second 0  3.5  1.5  1.5  1.5  2.5  4.5
       1  2.5  0.5  2.5  3.5  0.5  3.5
       2  3.5  0.5  2.5  1.5  2.5  1.5

Note that `concat` allows overlapping indexes. In order to avoid overlapping index the parameter `ignore_index` has to be set as `True`. 

In [6]:
pd.concat([df1, df2], ignore_index = True)

,a,b,c,d,e,f
0,-1.0,-2.0,4.0,3.0,NaN,NaN
1,3.0,4.0,-4.0,-3.0,NaN,NaN
2,2.0,-5.0,3.0,2.0,NaN,NaN
3,2.0,-5.0,3.0,2.0,NaN,NaN
4,3.5,1.5,1.5,1.5,2.5,4.5
5,2.5,0.5,2.5,3.5,0.5,3.5
6,3.5,0.5,2.5,1.5,2.5,1.5


And if we want to concatenate the DataFrames on a different axis:

In [7]:
concatenatedaxis = pd.concat([df1, df2], axis = 1, keys = ['ONE','TWO'])
concatenatedaxis

ONE           TWO                         
    a  b  c  d    f    e    d    c    b    a
0  -1 -2  4  3  4.5  2.5  1.5  1.5  1.5  3.5
1   3  4 -4 -3  3.5  0.5  3.5  2.5  0.5  2.5
2   2 -5  3  2  1.5  2.5  1.5  2.5  0.5  3.5
3   2 -5  3  2  NaN  NaN  NaN  NaN  NaN  NaN

## 2 Append

`append` is a shortcut for `concat` along `axis=0`:

In [8]:
d1 = pd.DataFrame(np.random.randint(10,99,(2,2)), index=[1001, 1002], columns=list('AB'))
d2 = pd.DataFrame(np.random.randint(10,99,(2,3)), index=[1002, 2001], columns=list('BCD'))

`append` allows overlapping index. Here we can see the two cases with `ignore_index` set respectively to true or false:

In [9]:
HTML(side_by_side2(d1, d2, d1.append(d2), d1.append(d2, ignore_index=True)))

## 3 Join

`join` is equivalent to the `sql` homonymous command. It's more advanced than `append` because has the parameter `how` that specifies how to merge indexes:

* `left` keeps left indexes.
* `right` keeps right indexes.
* `inner` intersects indexes.
* `outer` makes union of indexes.

In [10]:
HTML(side_by_side2(d1, d2))

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [11]:
d6 = d1.join(d2, rsuffix='_r', how='left')
d7 = d1.join(d2, rsuffix='_r', how='right')
d8 = d1.join(d2.D, how='inner')
d9 = d1.join(d2.D, how='outer')
HTML(side_by_side2(d6, d7, d8, d9))

## 4 Merge

pandas provides a single function, `merge`, as the entry point for all standard database join operations between DataFrame objects:

    pd.merge(left, right, how=’left’, on=None, left_on=None, right_on=None,
             left_index=False, right_index=False, sort=True,
             suffixes=(’_x’, ’_y’), copy=True)
             
* `on`: Columns to join on. Must be found in both DataFrames
* `left_on`: Columns from the left DataFrame to use as keys
* `right_on`: same for right
* `left_index`: If True, use the index (row labels) from the left DataFrame join key(s)
* `right_index`: same for right
* `how`: One of ’left’, ’right’, ’outer’, ’inner’. Defaults to ’inner’.
* `suffixes`: A tuple of string suffixes to apply to overlapping columns. Defaults to (’_x’, ’_y’).

In [12]:
left = pd.DataFrame({'key': ['foo', 'foo'], 'lval': [1, 2]})
right = pd.DataFrame({'key': ['foo', 'bar'], 'rval': [4, 5]})
HTML(side_by_side2(left, right))

,key,lval
0,foo,1
1,foo,2
,key,rval
0,foo,4
1,bar,5


In [13]:
pd.merge(left, right, on='key', how='right')

,key,lval,rval
0,foo,1,4
1,foo,2,4
2,bar,NaN,5


A practical example: revenues is a little dataset that we want to join with the 'cities' database to find the geographical location of the offices:

In [14]:
revenues = read_csv('example_data/p05_d2.txt', index_col=0)
cities = read_csv('example_data/cities.csv',
                  names=['City', 'State', 'lat', 'lon'])
HTML(side_by_side2(revenues, cities.head(8)))

IOError: File example_data/cities.csv does not exist

The 'cities' database contains 30101 records:

In [15]:
cities['City'].count()

NameError: name 'cities' is not defined

First we map a normalization function to standardize the 'City' and 'State' names. **`map`** applies the function passed as parameter to all elements of the `DataFrame`:

In [16]:
revenues = revenues.reindex(columns=['Name','Revenues','City','State'])
revenues['City'] = revenues['City'].map(lambda str: str.upper().strip())
revenues['State'] = revenues['State'].map(lambda str: str.upper().strip())
display(revenues)

,Name,Revenues,City,State
0,Roy,1250,NEW YORK,NY
1,Johnn,840,FRISCO,CA
2,Jim,349,HOUSTON,TX
3,Paul,1100,TAFT,OK
4,Ross,900,VENICE,IL


In [17]:
leftjoin = pd.merge(revenues, cities, left_on=['City', 'State'],
                                      right_on=['City', 'State'], how='left')
leftjoin = leftjoin.sort(columns='Revenues', ascending=False)
HTML(side_by_side2(revenues, leftjoin))

NameError: name 'cities' is not defined

## 5 Pivoting

`pivot_table` is useful to analize data in the so-called "stacked" or "record" format: here each field of each record are stored in colums and similar fields could have multiple duplicates:

In [18]:
data = read_csv('example_data/pd06_pivot_Example.txt')
data

,Unnamed: 0,date,variable,value
0,0,2000-01-03 00:00:00,A,0.397018
1,1,2000-01-04 00:00:00,A,0.621109
2,2,2000-01-05 00:00:00,A,-2.508284
3,3,2000-01-03 00:00:00,B,0.219796
4,4,2000-01-04 00:00:00,B,1.371027
5,5,2000-01-05 00:00:00,B,-1.005504
6,6,2000-01-03 00:00:00,C,-0.391149
7,7,2000-01-04 00:00:00,C,1.209346
8,8,2000-01-05 00:00:00,C,-0.059617
9,9,2000-01-03 00:00:00,D,-0.464558


In [19]:
pivot = data.pivot(index='date', columns='variable', values='value')
pivot.head()

variable,A,B,C,D
date,,,,
2000-01-03 00:00:00,0.397018,0.219796,-0.391149,-0.464558
2000-01-04 00:00:00,0.621109,1.371027,1.209346,-0.173605
2000-01-05 00:00:00,-2.508284,-1.005504,-0.059617,-1.649632


A practical example: apply pivoting to a dataset containing the statistics of te US crimes State by State, downloaded from the Stanford University "Human-Computer Interaction" group (http://hci.stanford.edu/jheer/workshop/data/):

In [20]:
data2 = read_csv('example_data/CrimeStatebyState.csv')
data2.ix[0:7000:1000]

IOError: File example_data/CrimeStatebyState.csv does not exist

Definition:
    pd.pivot_table(data, values=None, rows=None, cols=None, aggfunc='mean',
                   fill_value=None, margins=False, dropna=True)

* `values`: Columns to aggregate
* `aggfunc`: Default is `np.mean`. This can be a list of functions as well
* `aggfunc`: Boolean, add subtotals, grandtotals
* `dropna`: Exclude columns with NaN's

In [21]:
pd.set_option("display.colheader_justify","right")
pivot2 = pd.pivot_table(data2, values='Count',
                        index=['Year'],
                        columns=['State'],
                        margins=True, aggfunc=np.sum)
pivot2.ix[-10:,-8:]

NameError: name 'data2' is not defined

In [22]:
pivot3 = pd.pivot_table(data2, values='Count',
                        index=['Year', 'Type of Crime'],
                        columns=['State'],
                        margins=True, aggfunc=np.sum)
pivot3.ix[-10:,:8]

NameError: name 'data2' is not defined

If we want to select crimes for a given year and/or location:

In [23]:
pivot2.ix[2004:2005,:8]

NameError: name 'pivot2' is not defined

## 6 Stack and Unstack

`stack` “pivot” a level of the column labels to an inner-most row level. In the following case, since after stacking there are no column labels remaining, `stack` returns a Series instead of a Dataframe:

In [24]:
pivot3.ix[:5,:8]

NameError: name 'pivot3' is not defined

In [25]:
pivot3.stack()[:10]

NameError: name 'pivot3' is not defined

`unstack` reverse the operation by pivoting a row index on columns:

In [26]:
unstk = pivot3.unstack(1)
unstk.ix[:5,:8]

NameError: name 'pivot3' is not defined

---

Visit [www.add-for.com](<http://www.add-for.com/IT>) for more tutorials and updates.

This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.